<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/tools/OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnDemandLoaderTool Tutorial

Our `OnDemandLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `OnDemandLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-readers-wikipedia

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

### Define Tool

We first define the `WikipediaReader`. Note that the `load_data` interface to `WikipediaReader` takes in a list of `pages`. By default, this queries the Wikipedia search endpoint which will autosuggest the relevant pages.

We then wrap it into our `OnDemandLoaderTool`.

By default since we don't specify the `index_cls`, a simple vector store index is initialized.

In [ ]:
reader = WikipediaReader()

In [ ]:
tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

#### Testing

We can try running the tool by itself (or as a LangChain tool), just to showcase what the interface is like! 

Note that besides the arguments required for the data loader, the tool also takes in a `query_str` which will be
the query against the index.

In [ ]:
# run tool by itself
tool(["Berlin"], query_str="What's the arts and culture scene in Berlin?")

"\nBerlin has a vibrant and diverse arts and culture scene. It is home to 44 theaters and stages, three major opera houses, and numerous art galleries. The cityscape of Berlin displays large quantities of urban street art, and the Berlin Wall has become one of the largest open-air canvasses in the world. Berlin also has a long history of gay culture, and is an important birthplace of the LGBT rights movement. There are many festivals and events throughout the year, such as the Berlin International Film Festival, the Karneval der Kulturen, the Berlin Festival, and the New Year's Eve celebrations. The city is also home to many museums, such as the Museum Island, the Gemäldegalerie, the Neue Nationalgalerie, the Pergamon Museum, the Bode Museum, the Hamburger Bahnhof, the German Museum of Technology, the Jewish Museum, the Museum für Naturkunde, the Kupferstichkabinett Berlin, the Museum Berggruen, and the Beate Uhse Erotic Museum."

In [ ]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)

In [ ]:
lc_tool.run(
    tool_input={
        "pages": ["Berlin"],
        "query_str": "What's the arts and culture scene in Berlin?",
    }
)

### Initialize LangChain Agent

For tutorial purposes, the agent just has access to one tool - the Wikipedia Reader

Note that we need to use Structured Tools from LangChain.

In [ ]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)

In [ ]:
agent = initialize_agent(
    [lc_tool],
    llm=llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=True,
)

# Now let's run some queries! 

The OnDemandLoaderTool allows the agent to simultaneously 1) load the data from Wikipedia, 2) query that data.

In [ ]:
agent.run("Tell me about the arts and culture of Berlin")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["Culture in Berlin"],
    "query_str": "What is the arts and culture scene like in Berlin?"
  }
}
```

Observation: 
The arts and culture scene in Berlin is vibrant and diverse. The city is home to over 600 art galleries, 153 museums, and numerous cultural institutions. It is a world city of culture and creative industries, and is home to many international and regional television and radio stations. Berlin is also home to two major German-language publishing houses, and is an important center of the European and German film industry. The city is also known for its nightlife, with many clubs and festivals, such as the Berlin International Film Festival, the Karneval der Kulturen, and the Christopher Street Day. Berlin is also home to the largest gay fetish festivals in Europe.
Thought:The human may want more specific information about certain aspects of Berlin's arts an

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "Dance, Dance, Dance" may refer to: 
"Dance, Dance, Dance" (The Beach Boys song)
"Dance, Dance, Dance" (Neil Young song)
"Dance, Dance, Dance" (Yowsah, Yowsah, Yowsah)
"Dance Dance Dance" (James Cottriall song)
"Dance Dance Dance" (E-girls song)
Dance Dance Dance/My Lady
soundtrack
Why Do You Have to Go/Dance, Dance, Dance
Youth Novels
Fly Like an Eagle
Dance Dance Dance (German TV series)
Dance Dance Dance (British TV series)
Dance Dance Dance (novel)
Dance, Dance, Dance: The Best of Chic
Dance, Dance (disambiguation)

In [ ]:
agent.run("Tell me about the critical reception to The Departed")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["The Departed"],
    "query_str": "critical reception"
  }
}
```


Observation: 
The critical reception of The Departed was overwhelmingly positive. On review aggregator Rotten Tomatoes, the film holds a 91% approval rating based on 284 reviews, with an average rating of 8.3/10. The website's critics consensus reads, "Featuring outstanding work from an excellent cast, The Departed is a thoroughly engrossing gangster drama with the gritty authenticity and soupy morality we have come to expect from Martin Scorsese." Metacritic, which uses a weighted average, assigned the film a score of 85 out of 100 based on 39 critics, indicating "universal acclaim". Audiences polled by CinemaScore gave the film an average grade of "A−" on an A+ to F scale. Entertainment Weekly ranked it on its end-of-the-decade "Best of" list, saying: "If they're lucky, directors make one classic film 

"The critical reception to The Departed was overwhelmingly positive, with an approval rating of 91% on Rotten Tomatoes and a score of 85 out of 100 on Metacritic. It was praised for its outstanding cast, gritty authenticity, and soupy morality. Many critics ranked it alongside Scorsese's past successes, including Taxi Driver, Raging Bull, and Goodfellas."